# Pip

In [183]:
%pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install "unstructured[pdf]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/17.9 MB ? eta -:--:--Downloading onnx-1.20.1-cp312-abi3-macosx_12_0_universal2.whl (17.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/529.1 kB ? eta -:--:--Downloading effdet-0.4.1-py3-none-any.whl (112 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
%pip install langchain_unstructured

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install neo4j langchain-neo4j

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -U langchain langchain-core langchain-openai langchain-community pydantic

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.1 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached multidict-6.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached yarl-1.22.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (75 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached aiohttp-3.13.2-cp312-cp312-macosx_11_0_arm64.whl (491 kB)
Using cached dataclasses_json-0.

In [ ]:
%pip install --upgrade --force-reinstall gensim numpy

In [ ]:
%pip install -U sentence-transformers transformers flash-attn

In [ ]:
%pip uninstall -y bottleneck numexpr
%pip install --upgrade pandas pyarrow numpy>=2.0

In [ ]:
%pip install "numpy<2.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.10.1 requires numexpr>=2.6.2, which is not installed.
open-webui 0.6.43 requires pillow==12.0.0, but you have pillow 11.3.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", 

In [ ]:
%pip install langchain-milvus langchain-community

In [ ]:
%pip install fastapi uvicorn nest-asyncio

# Function Calls

In [1]:
NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "manav@1234"

EMBED_URL = "http://127.0.0.1:8001/v1/embeddings"
EMBED_MODEL = "jinaai/jina-embeddings-v3"
VECTOR_DIMENSION = 1024 

LLM_URL = "http://127.0.0.1:8000/v1/chat/completions"
LLM_MODEL = "mistralai/Ministral-3-14B-Reasoning-2512"

In [2]:
from ingestor import SmartURLIngestor
from db_manager import Neo4jManager

In [ ]:
# CAUTION: Clears the dataset empty

# manager = Neo4jManager(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# manager.hard_reset()

Data Cleared: Deleted 87 nodes and 0 relationships.
Schema Cleared: All constraints and indexes removed.
Hard Reset Complete: The database is now empty and has no rules.


In [6]:
db_config = {
    "uri": NEO4J_URI,
    "user": NEO4J_USER,
    "password": NEO4J_PASSWORD
}

crawler = SmartURLIngestor(db_config, EMBED_URL, EMBED_MODEL, LLM_URL, LLM_MODEL, Neo4jManager)

crawler.ingest_domain("https://www.bom.gov.au/")


🚜 Starting Scrapy for domain: bom.gov.au
   ♻️  Wiping old data for bom.gov.au...


2026-02-09 23:22:47 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2026-02-09 23:22:47 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.13.1, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ], pyOpenSSL 24.2.1 (OpenSSL 3.0.18 30 Sep 2025), cryptography 43.0.0, Platform macOS-26.1-arm64-arm-64bit
2026-02-09 23:22:47 [scrapy.addons] INFO: Enabled addons:
[]
2026-02-09 23:22:47 [py.warnings] WARNING: /opt/anaconda3/lib/python3.12/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation

🚀 INGESTING: https://www.bom.gov.au/sites/default/files/2023-01/water-information-urban-water-management-information-requirements-v1.1-2018.pdf
   🔍 Analyzing Layout: water-information-urban-water-management-information-requirements-v1.1-2018.pdf...


2026-02-09 23:22:53 [pikepdf._core] INFO: pikepdf C++ to Python logger bridge initialized


2026-02-09 23:22:56 [unstructured_inference] INFO: Reading PDF for file: /var/folders/w4/207pk0bd1k932myh93lvc6jr0000gn/T/tmpu2s_ysb6/document ...
2026-02-09 23:23:30 [scrapy.crawler] INFO: Received SIGINT, shutting down gracefully. Send again to force 


KeyboardInterrupt: 

⚠️ Metadata Extraction Failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x354aa8cb0>: Failed to establish a new connection: [Errno 61] Connection refused'))
   Found Metadata: ('Unknown Title', [], [], 0, None)
   ❌ Embedding Error: HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /v1/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x365d768a0>: Failed to establish a new connection: [Errno 61] Connection refused'))


2026-02-09 23:23:36 [scrapy.core.engine] INFO: Closing spider (shutdown)


🚀 INGESTING: https://www.bom.gov.au/sites/default/files/2025-11/water-information-register-of-changes-persons-and-commercially-sensitive-sites-v1-2025.pdf
   🔍 Analyzing Layout: water-information-register-of-changes-persons-and-commercially-sensitive-sites-v1-2025.pdf...


In [7]:
manager = Neo4jManager(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
def print_crawl_stats(db_manager):
    with db_manager.driver.session() as session:
        # Count Pages
        count = session.run("MATCH (p:WebPage) RETURN count(p) as c").single()['c']
        print(f"📊 Total Web Pages Ingested: {count}")
        
        # Count PDFs
        pdf_count = session.run("MATCH (d:Document {source_type: 'web'}) RETURN count(d) as c").single()['c']
        print(f"📄 Total Web PDFs Ingested: {pdf_count}")

        # List Domains
        domains = session.run("MATCH (p:WebPage) RETURN DISTINCT p.domain as d").data()
        print(f"🌐 Domains: {[d['d'] for d in domains]}")

# Run it
print_crawl_stats(manager)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: source_type)} {position: line: 1, column: 20, offset: 19} for query: "MATCH (d:Document {source_type: 'web'}) RETURN count(d) as c"


📊 Total Web Pages Ingested: 81
📄 Total Web PDFs Ingested: 0
🌐 Domains: ['bom.gov.au']


2026-02-09 23:23:44 [unstructured_inference] INFO: Reading PDF for file: /var/folders/w4/207pk0bd1k932myh93lvc6jr0000gn/T/tmpxs3wrtru/document ...


In [8]:
def list_all_ingested_urls(db_manager):
    """
    Fetches and prints ALL ingested WebPage URLs from the database.
    """
    query = """
    MATCH (p:WebPage)
    RETURN p.url as url, p.title as title
    ORDER BY p.last_crawled DESC
    """
    
    print("🔍 Fetching all ingested URLs...")
    
    with db_manager.driver.session() as session:
        results = session.run(query)
        count = 0
        
        print("\n--- All Ingested URLs ---")
        for record in results:
            count += 1
            print(f"🔗 {record['url']}")
            # Optional: print title if you want
            # print(f"   Title: {record['title']}") 
            
        print(f"\n✅ Total URLs found: {count}")

# Usage:
list_all_ingested_urls(manager)

🔍 Fetching all ingested URLs...

--- All Ingested URLs ---
🔗 https://www.sws.bom.gov.au/?ref=ftr
🔗 https://www.bom.gov.au/australia/meteye/?ref=ftr
🔗 https://www.bom.gov.au/aviation/about-us/faq/
🔗 https://www.bom.gov.au/aviation/contact-us/
🔗 https://www.bom.gov.au/aviation/warnings/graphical-sigmet/
🔗 https://www.bom.gov.au/aviation/
🔗 https://www.bom.gov.au/ant/observations/map.shtml
🔗 https://reg.bom.gov.au/climate/data-services/education.shtml
🔗 https://www.bom.gov.au/water/regulations/search.php
🔗 https://www.bom.gov.au/water/ssf/?ref=ftr
🔗 https://www.bom.gov.au/water/dashboards/
🔗 https://www.bom.gov.au/rss/?ref=ftr
🔗 https://www.bom.gov.au/weather-services/?ref=ftr
🔗 https://www.bom.gov.au/cgi-bin/climate/change/trendmaps.cgi
🔗 https://www.bom.gov.au/aviation/?ref=ftr
🔗 https://www.bom.gov.au/glossary?f%5B0%5D=type%3A100816
🔗 https://www.bom.gov.au/bom-weather-app/weather-app-help-and-support?utm_source=beta.bom.gov.au%2Fresources%2Flearn-and-explore%2Ffire-weather-knowledge-c

⚠️ Metadata Extraction Failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x355475ca0>: Failed to establish a new connection: [Errno 61] Connection refused'))
   Found Metadata: ('Unknown Title', [], [], 0, None)
   ❌ Embedding Error: HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /v1/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x365d4f1d0>: Failed to establish a new connection: [Errno 61] Connection refused'))


2026-02-09 23:26:16 [scrapy.extensions.logstats] INFO: Crawled 91 pages (at 91 pages/min), scraped 83 items (at 83 items/min)
2026-02-09 23:26:16 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 47926,
 'downloader/request_count': 126,
 'downloader/request_method_count/GET': 126,
 'downloader/response_bytes': 4534737,
 'downloader/response_count': 126,
 'downloader/response_status_count/200': 90,
 'downloader/response_status_count/301': 34,
 'downloader/response_status_count/401': 1,
 'downloader/response_status_count/404': 1,
 'dupefilter/filtered': 6645,
 'elapsed_time_seconds': 208.322422,
 'finish_reason': 'shutdown',
 'finish_time': datetime.datetime(2026, 2, 9, 12, 26, 16, 432822, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 14759565,
 'httpcompression/response_count': 90,
 'item_scraped_count': 89,
 'log_count/INFO': 15,
 'log_count/WARNING': 1,
 'memusage/max': 1696694272,
 'memusage/startup': 624115712,
 'request_depth_max'

# Create Index

In [ ]:
import time
from neo4j import GraphDatabase

def create_index():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    
    with driver.session() as session:
        print("⚙️ Checking Vector Index...")
        
        # 1. Check if it exists
        indexes = session.run("SHOW INDEXES").data()
        index_exists = any(i['name'] == 'chunk_vector_index' for i in indexes)
        
        if index_exists:
            session.run("DROP INDEX chunk_vector_index")
            print("🗑️ Old index dropped.")

        # 2. Create it
        print(f"   Creating 'chunk_vector_index' with {VECTOR_DIMENSION} dimensions...")
        session.run(f"""
        CREATE VECTOR INDEX chunk_vector_index IF NOT EXISTS
        FOR (c:Chunk) ON (c.embedding)
        OPTIONS {{indexConfig: {{
         `vector.dimensions`: {VECTOR_DIMENSION},  
         `vector.similarity_function`: 'cosine'
        }}}}
        """)
        session.run(f"""
        CREATE VECTOR INDEX document_vector_index IF NOT EXISTS
        FOR (d:Document) ON (d.embedding)
        OPTIONS {{indexConfig: {{
         `vector.dimensions`: {VECTOR_DIMENSION},  
         `vector.similarity_function`: 'cosine'
        }}}}
        """)
        
        # 3. Wait for it to come Online
        print("   Waiting for index to come ONLINE...", end="")
        while True:
            status = session.run("SHOW INDEXES YIELD name, state, populationPercent WHERE name = 'chunk_vector_index' RETURN state, populationPercent").single()
            if status and status['state'] == "ONLINE":
                print(" ✅ Done!")
                break
            elif status and status['state'] == "FAILED":
                print(" ❌ Index Creation Failed! Check Neo4j logs.")
                break
            else:
                percent = status['populationPercent'] if status else 0
                print(f". ({percent:.0f}%)", end="", flush=True)
                time.sleep(1)

    driver.close()

create_index()

⚙️ Checking Vector Index...
   Creating 'chunk_vector_index' with 1024 dimensions...
   Waiting for index to come ONLINE.... (0%) ✅ Done!
